In [ ]:
import spacy
import os
import numpy as np

nlp = spacy.load('en')

In [ ]:
labels = []
for f in os.listdir(os.path.join("data","train")):
    path = os.path.join("data","train",f)
    if os.path.isfile(path):
        labels.append(path)

train_doc_vecs = []
train_Y = []
for label_file in labels:
    sentences = open(label_file, "r").readlines()
    for sentence in sentences:
        if len(sentence.lstrip().rstrip()) > 0:
            vec_seq = []
            for token in nlp(unicode(sentence)):
                vec_seq.append(token.vector)
            train_vec_seqs.append(np.array(vec_seq))
            train_Y.append(int(label_file.split("label_")[1].split(".txt")[0]))
                        

In [ ]:
#make sequences equal length
maxlen = 40
new_seqs = []
for vec_seq in train_vec_seqs:
    orig_len, vec_len = np.shape(vec_seq)
    if orig_len < maxlen:
        new = np.zeros((maxlen,vec_len))
        new[maxlen-orig_len:,:] = vec_seq
    else:
        new = sequence[orig_len-maxlen:,:]
    new_seqs.append(new)
    
train_X = new_seqs

In [ ]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

train_X = train_vec_seqs

x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.1)

y_train, y_test = [np_utils.to_categorical(x,nb_classes=4) for x in (y_train,y_test)]

In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout, GRU, LSTM,  Input, merge

ip = Input(shape=(maxlen,300), dtype='float32')
normal = LSTM(100,dropout_W=0.1,dropout_U=0.1)(ip)
reverse = LSTM(100,dropout_W=0.1,dropout_U=0.1,go_backwards=True)(ip)
merged = merge([normal, reverse], mode='concat', concat_axis=-1)
dropout = Dropout(0.1)(merged)
output = Dense(4, activation='softmax')(dropout)
model = Model(input=ip, output=output)
model.compile('adam','categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(np.array(x_train), y_train, batch_size=20,nb_epoch = 6)

In [ ]:
scores = model.evaluate(np.array(x_test), y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#save model

model_json = model.to_json()
jf = open("model.json","w")
jf.write(model_json)
model.save_weights("model.h5")


In [ ]:
def transform_input(input_str):
    doc = nlp(unicode(input_str))
    vec_seq = []
    for token in doc:
        vec_seq.append(token.vector)
    orig_len, vec_len = np.shape(vec_seq)
    new = []
    if orig_len < maxlen:
        new = np.zeros((maxlen,vec_len))
        new[maxlen-orig_len:,:] = vec_seq
    else:
        new = vec_seq[orig_len-maxlen:,:]
    return np.array([new])

model.predict(np.array(transform_input("enlighten me about the computer stats")))

In [ ]:
from keras.models import model_from_json
debug = True
def load_model(mpath="",wpath=""):
	if mpath=="" and wpath=="":
		mpath = "data/model.json"
		wpath = "data/model.h5"
	# load json and create model
	json_file = open(mpath, 'r')
	model_json = json_file.read()
	json_file.close()
	model = model_from_json(model_json)	
	# load weights into new model
	model.load_weights(wpath)

	if debug:
		print "Model loaded. Compiling..."
	model.compile('adam','categorical_crossentropy')

	if debug:
		print "Model compiled."
		print model.summary()

	return model

In [ ]:
r = load_model()

In [ ]:
from keras.utils.visualize_util import plot
plot(r, to_file='model.png')

In [ ]:
import numpy as np
import spacy
nlp = spacy.load("en")
def transform_input(input_str):
    doc = nlp(unicode(input_str))
    vec_seq = []
    for token in doc:
        vec_seq.append(token.vector)
    orig_len, vec_len = np.shape(vec_seq)
    new = []
    if orig_len < maxlen:
        new = np.zeros((maxlen,vec_len))
        new[maxlen-orig_len:,:] = vec_seq
    else:
        new = vec_seq[orig_len-maxlen:,:]
    return np.array([new])

r.predict(np.array(transform_input("enlighten me about the computer stats")))